In [46]:
######### Takes in the raw data game data just after the scrape and creates yearly summary stats for team


import pandas as pd
import numpy as np
import os
import ast # for literal_eval function



In [47]:

# Load Data
conference_df = pd.read_csv('../data/yearly_conference_members.csv')
games_df = pd.read_csv('../data/cfb_scrape_raw.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})



# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


C:\Users\Justin\AppData\Local\Temp\ipykernel_18228\4288072279.py:3: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('../data/cfb_scrape_raw.csv')


In [48]:
### Extract the ranking from the team name where applicable
## Pool ranking is contained in parentheses at the beginning of the team name string of the Winner and Loser columns
def extract_ranking(team_name):
    """Extract the ranking from the team name where applicable."""
    # If the team name does not start with a parenthesis, there is no ranking
    if team_name[0] != '(':
        return None
    
    # Find the closing parenthesis
    closing_paren = team_name.find(')')
    
    # Extract the ranking
    ranking = team_name[1:closing_paren]
    
    return ranking

# Apply function to Winner column and save in new 'Winner_Rank' column
games_df['Winner_Rank'] = games_df['Winner'].apply(extract_ranking)
# Apply function to Loser column and save in new 'Loser_Rank' column
games_df['Loser_Rank'] = games_df['Loser'].apply(extract_ranking)

# Remove the ranking from the team name but only from the front of the string
games_df['Winner'] = games_df['Winner'].str.replace(r'^\(\d+\)\s', '')
games_df['Loser'] = games_df['Loser'].str.replace(r'^\(\d+\)\s', '')

# Strip leading and trailing whitespace from team names
games_df['Winner'] = games_df['Winner'].str.strip()
games_df['Loser'] = games_df['Loser'].str.strip()

C:\Users\Justin\AppData\Local\Temp\ipykernel_18228\991721257.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  games_df['Winner'] = games_df['Winner'].str.replace(r'^\(\d+\)\s', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_18228\991721257.py:24: FutureWarning: The default value of regex will change from True to False in a future version.
  games_df['Loser'] = games_df['Loser'].str.replace(r'^\(\d+\)\s', '')


In [49]:
conference_df = pd.read_csv('../data/yearly_conference_members.csv')

# rename Unnamed: 0 to year in conference_df
conference_df = conference_df.rename(columns={'Unnamed: 0': 'Year'})

# Convert string representations of lists back into actual lists
for col in conference_df.columns[1:]:
    conference_df[col] = conference_df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


# Function to retrieve the conference based on year and team name
def get_conference(year, team_name, conference_lookup):
    """
    Given a year and team name, retrieve the conference the team belongs to.
    If no conference found, return "Independent/Unknown".
    """
    return conference_lookup.get((year, team_name), "Independent/Unknown")

# Create a lookup dictionary for team-year to conference mapping
conference_lookup = {}

for _, row in conference_df.iterrows():
    year = row['Year']
    for col, teams in row[1:].items():
        for team in teams:
            conference_lookup[(year, team)] = col

# Use funtion to assign conference affiliation the teams in each row but only if 
# the team name is not null
games_df['winner_conference'] = games_df.apply(lambda row: get_conference(row['Season'], row['Winner'], conference_lookup) if pd.notna(row['Winner']) else None, axis=1)
games_df['loser_conference'] = games_df.apply(lambda row: get_conference(row['Season'], row['Loser'], conference_lookup) if pd.notna(row['Loser']) else None, axis=1)


### Create a new column for conference and non-conference games
# Function to determine if a game is a conference game
def determine_conference_game(conference1, conference2):
    if conference1 == conference2:
        return 'Conference'
    else:
        return 'Non-Conference'
    
# Apply to create new 'game_type' column using the determine_conference_game function and the winner and loser conference columns
games_df['game_type'] = games_df.apply(lambda row: determine_conference_game(row['winner_conference'], row['loser_conference']) if pd.notna(row['winner_conference']) and pd.notna(row['loser_conference']) else None, axis=1)

# Set game type to Non-Conferecne if either team is Independent/Unknown
games_df.loc[games_df['winner_conference'] == 'Independent/Unknown', 'game_type'] = 'Non-Conference'
games_df.loc[games_df['loser_conference'] == 'Independent/Unknown', 'game_type'] = 'Non-Conference'

# Rename the points columns to be more to match code down the line
# Pts = winner_pts, Pts.1 = loser_pts
games_df = games_df.rename(columns={'Pts': 'winner_pts', 'Pts.1': 'loser_pts'})

# season_year to season
games_df = games_df.rename(columns={'Season': 'season'})

# unnamed: 6 to loc_ind
games_df = games_df.rename(columns={'Unnamed: 6': 'loc_ind'})

# make all column names lowercase
games_df.columns = games_df.columns.str.lower()

In [50]:
# Rename the dataframe to df to patch together the two wookbooks
df = games_df

# Load data and initial transformations

df['point_diff'] = df['winner_pts'] - df['loser_pts']

# Extract data for winners and losers
winner_data = df[['season', 'winner', 'winner_conference', 'loc_ind', 'game_type', 'point_diff', 'winner_pts', 'loser_pts']].copy()
winner_data.columns = ['season', 'team', 'conference', 'location', 'game_type', 'point_diff', 'team_pts', 'opponent_pts']
winner_data['result'] = 'win'

loser_data = df[['season', 'loser', 'loser_conference', 'loc_ind', 'game_type', 'point_diff', 'loser_pts', 'winner_pts']].copy()
loser_data.columns = ['season', 'team', 'conference', 'location', 'game_type', 'point_diff', 'team_pts', 'opponent_pts']
loser_data['result'] = 'loss'
loser_data['point_diff'] = -loser_data['point_diff']

game_data = pd.concat([winner_data, loser_data], axis=0).reset_index(drop=True)

# Handling ties
ties = df[df['point_diff'] == 0][['season', 'winner', 'winner_conference', 'loc_ind', 'game_type', 'point_diff', 'winner_pts', 'loser_pts']].copy()
ties.columns = ['season', 'team', 'conference', 'location', 'game_type', 'point_diff', 'team_pts', 'opponent_pts']
ties['result'] = 'tie'
game_data = pd.concat([game_data, ties], axis=0).reset_index(drop=True)

# Define function to calculate stats
def calculate_stats(data, prefix=''):
    stats = data.groupby(['season', 'team', 'result']).size().unstack(fill_value=0)
    stats.columns = [prefix + col for col in stats.columns]
    stats.reset_index(inplace=True)
    
    # Calculate win percentage
    total_games = stats[[col for col in stats.columns if col.endswith(('win', 'loss', 'tie'))]].sum(axis=1)
    stats[prefix+'win_percentage'] = (stats.get(prefix+'win', 0) / total_games).round(4)
    
    return stats

# Calculate stats for various situations
overall_stats = calculate_stats(game_data, 'overall_')
home_stats = calculate_stats(game_data[game_data['location'].isna()], 'home_')
away_stats = calculate_stats(game_data[game_data['location'] == '@'], 'away_')
conference_stats = calculate_stats(game_data[game_data['conference'] == game_data['game_type']], 'conference_')
non_conference_stats = calculate_stats(game_data[game_data['conference'] != game_data['game_type']], 'non_conference_')
blowout_stats = calculate_stats(game_data[game_data['point_diff'].abs() >= 21], 'blowout_')
close_stats = calculate_stats(game_data[game_data['point_diff'].abs() <= 7], 'close_')

# Merging all stats to form the final dataframe
team_conference = game_data[['season', 'team', 'conference']].drop_duplicates()
final_df = pd.merge(team_conference, overall_stats, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, home_stats, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, away_stats, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, conference_stats, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, non_conference_stats, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, blowout_stats, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, close_stats, on=['season', 'team'], how='outer')

# Filling NaN values with 0
final_df = final_df.fillna(0)

final_df.head()


,season,team,conference,overall_loss,overall_tie,overall_win,overall_win_percentage,home_loss,home_win,home_win_percentage,...,non_conference_tie,non_conference_win,non_conference_win_percentage,blowout_loss,blowout_win,blowout_win_percentage,close_loss,close_tie,close_win,close_win_percentage
0,1869,Rutgers,Independent/Unknown,1,0,1,0.5,1.0,1.0,0.5,...,0,1,0.5,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1869,Princeton,Independent/Unknown,1,0,1,0.5,1.0,1.0,0.5,...,0,1,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1870,Rutgers,Independent/Unknown,1,0,1,0.5,1.0,1.0,0.5,...,0,1,0.5,0.0,0.0,0.0,1.0,0.0,1.0,0.5
3,1870,Princeton,Independent/Unknown,0,0,1,1.0,0.0,1.0,1.0,...,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,1872,Rutgers,Independent/Unknown,1,1,2,0.5,1.0,1.0,0.5,...,1,2,0.5,0.0,0.0,0.0,1.0,1.0,2.0,0.5


In [51]:
# Define function to calculate points metrics
def calculate_points(data, prefix=''):
    total_points_scored = data.groupby(['season', 'team'])['team_pts'].sum().reset_index()
    total_points_scored.columns = ['season', 'team', prefix + 'total_pts_scored']
    
    total_points_allowed = data.groupby(['season', 'team'])['opponent_pts'].sum().reset_index()
    total_points_allowed.columns = ['season', 'team', prefix + 'total_pts_allowed']
    
    avg_points_scored = data.groupby(['season', 'team'])['team_pts'].mean().reset_index()
    avg_points_scored.columns = ['season', 'team', prefix + 'avg_pts_scored']
    
    avg_points_allowed = data.groupby(['season', 'team'])['opponent_pts'].mean().reset_index()
    avg_points_allowed.columns = ['season', 'team', prefix + 'avg_pts_allowed']
    
    merged = pd.merge(total_points_scored, total_points_allowed, on=['season', 'team'], how='outer')
    merged = pd.merge(merged, avg_points_scored, on=['season', 'team'], how='outer')
    merged = pd.merge(merged, avg_points_allowed, on=['season', 'team'], how='outer')
    
    return merged

# Calculate points metrics for various situations
overall_points = calculate_points(game_data, 'overall_')
home_points = calculate_points(game_data[game_data['location'].isna()], 'home_')
away_points = calculate_points(game_data[game_data['location'] == '@'], 'away_')
conference_points = calculate_points(game_data[game_data['conference'] == game_data['game_type']], 'conference_')
non_conference_points = calculate_points(game_data[game_data['conference'] != game_data['game_type']], 'non_conference_')
blowout_points = calculate_points(game_data[game_data['point_diff'].abs() >= 21], 'blowout_')
close_points = calculate_points(game_data[game_data['point_diff'].abs() <= 7], 'close_')

# Merging points metrics to the final dataframe
final_df = pd.merge(final_df, overall_points, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, home_points, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, away_points, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, conference_points, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, non_conference_points, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, blowout_points, on=['season', 'team'], how='outer')
final_df = pd.merge(final_df, close_points, on=['season', 'team'], how='outer')

# Filling NaN values with 0
final_df = final_df.fillna(0)

final_df.head()


,season,team,conference,overall_loss,overall_tie,overall_win,overall_win_percentage,home_loss,home_win,home_win_percentage,...,non_conference_avg_pts_scored,non_conference_avg_pts_allowed,blowout_total_pts_scored,blowout_total_pts_allowed,blowout_avg_pts_scored,blowout_avg_pts_allowed,close_total_pts_scored,close_total_pts_allowed,close_avg_pts_scored,close_avg_pts_allowed
0,1869,Rutgers,Independent/Unknown,1,0,1,0.5,1.0,1.0,0.5,...,3.0,6.00,0.0,0.0,0.0,0.0,6.0,4.0,6.0,4.00
1,1869,Princeton,Independent/Unknown,1,0,1,0.5,1.0,1.0,0.5,...,6.0,3.00,0.0,0.0,0.0,0.0,4.0,6.0,4.0,6.00
2,1870,Rutgers,Independent/Unknown,1,0,1,0.5,1.0,1.0,0.5,...,4.0,4.50,0.0,0.0,0.0,0.0,8.0,9.0,4.0,4.50
3,1870,Princeton,Independent/Unknown,0,0,1,1.0,0.0,1.0,1.0,...,6.0,2.00,0.0,0.0,0.0,0.0,6.0,2.0,6.0,2.00
4,1872,Rutgers,Independent/Unknown,1,1,2,0.5,1.0,1.0,0.5,...,2.0,2.25,0.0,0.0,0.0,0.0,8.0,9.0,2.0,2.25


In [52]:
final_df.sample(30)

,season,team,conference,overall_loss,overall_tie,overall_win,overall_win_percentage,home_loss,home_win,home_win_percentage,...,non_conference_avg_pts_scored,non_conference_avg_pts_allowed,blowout_total_pts_scored,blowout_total_pts_allowed,blowout_avg_pts_scored,blowout_avg_pts_allowed,close_total_pts_scored,close_total_pts_allowed,close_avg_pts_scored,close_avg_pts_allowed
8443,1962,Tulsa,Missouri Valley Conference,5,0,5,0.5000,3.0,4.0,0.5714,...,21.900000,18.100000,93.0,77.0,23.250000,19.250000,73.0,74.0,24.333333,24.666667
1026,1904,Cumberland,Independent/Unknown,1,0,1,0.5000,1.0,0.0,0.0000,...,13.500000,11.500000,27.0,5.0,27.000000,5.000000,0.0,0.0,0.000000,0.000000
14191,2004,Toledo,Mid-American Conference,4,0,9,0.6923,3.0,5.0,0.6250,...,33.230769,31.076923,201.0,229.0,33.500000,38.166667,43.0,45.0,21.500000,22.500000
2966,1922,Utah,Independent/Unknown,1,0,7,0.8750,1.0,5.0,0.8333,...,19.125000,5.125000,76.0,0.0,38.000000,0.000000,19.0,12.0,9.500000,6.000000
4060,1930,Colorado State,Independent/Unknown,6,0,3,0.3333,3.0,0.0,0.0000,...,7.444444,11.555556,26.0,39.0,13.000000,19.500000,7.0,21.0,2.333333,7.000000
20322,1926,Fordham,Independent/Unknown,1,0,0,0.0000,1.0,0.0,0.0000,...,13.000000,28.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
12301,1988,New Mexico State,Independent/Unknown,10,0,1,0.0909,5.0,0.0,0.0000,...,15.545455,34.272727,19.0,210.0,3.800000,42.000000,70.0,78.0,23.333333,26.000000
18968,1915,Morgan,Independent/Unknown,1,0,0,0.0000,1.0,0.0,0.0000,...,0.000000,57.000000,0.0,57.0,0.000000,57.000000,0.0,0.0,0.000000,0.000000
12277,1988,Virginia Tech,Independent/Unknown,8,0,3,0.2727,5.0,2.0,0.2857,...,16.000000,24.000000,102.0,171.0,17.000000,28.500000,34.0,42.0,17.000000,21.000000
20763,1930,Rhode Island,Independent/Unknown,1,0,0,0.0000,1.0,0.0,0.0000,...,0.000000,7.000000,0.0,0.0,0.000000,0.000000,0.0,7.0,0.000000,7.000000


In [54]:
final_df.columns

# save to csv
final_df.to_csv('../data/cfb_yearly_summary_stats.csv', index=False)